<a href="https://colab.research.google.com/github/harinijs03/2023103549_SDC_assignment/blob/main/Multimodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers torchaudio openai-whisper gradio pillow -q
!pip install torch torchvision -q
import gradio as gr
import whisper
from transformers import BlipProcessor, BlipForQuestionAnswering, pipeline
from PIL import Image
import torch

# Load Whisper model for audio transcription
asr_model = whisper.load_model("base")

# Load BLIP model + processor for image Q&A
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
blip_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# Load QA model for text-based Q&A (for audio transcript)
qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Step 1: Image-based Q&A
def image_based_qa(image, question):
    inputs = blip_processor(image, question, return_tensors="pt")
    with torch.no_grad():
        out = blip_model.generate(**inputs)
    answer = blip_processor.decode(out[0], skip_special_tokens=True)
    return answer

# Step 2: Audio-based transcription
def transcribe_audio(audio):
    if audio is None:
        return None
    result = asr_model.transcribe(audio)
    return result["text"]

# Step 3: Audio Q&A
def audio_based_qa(audio, question):
    transcript = transcribe_audio(audio)
    if not transcript:
        return "No audio or empty transcript."
    answer = qa_model(question=question, context=transcript)
    return f"Answer: {answer['answer']}\n\nTranscript:\n{transcript}"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Multimodal Question Answering (Image + Audio)")

    with gr.Tab("Image Q&A"):
        gr.Markdown("### Upload an image and ask a question about it")
        image_input = gr.Image(type="pil", label="📸 Upload an image")
        question_image = gr.Textbox(label="❓ Ask a question about the image")
        answer_image = gr.Textbox(label="💡 Answer", lines=3)
        btn_image = gr.Button("Submit")
        btn_image.click(fn=image_based_qa, inputs=[image_input, question_image], outputs=answer_image)

    with gr.Tab("Audio Q&A"):
        gr.Markdown("### Upload an audio file and ask a question about it")
        audio_input = gr.Audio(type="filepath", label="🎧 Upload an audio file")
        question_audio = gr.Textbox(label="❓ Ask a question about the audio")
        answer_audio = gr.Textbox(label="💡 Answer", lines=5)
        btn_audio = gr.Button("Submit")
        btn_audio.click(fn=audio_based_qa, inputs=[audio_input, question_audio], outputs=answer_audio)

demo.launch()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c7172592ac62d83ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
